In [1]:
import numpy as np
import datajoint as dj
import time
import pymeshfix
import os
import datetime
import calcification_Module as cm

#for supressing the output
import os, contextlib
import pathlib
import subprocess

#for error counting
from collections import Counter

#for reading in the new raw_skeleton files
import csv

from meshparty import trimesh_io

#for filtering
import math
from pykdtree.kdtree import KDTree


In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


Connecting celiib@10.28.0.34:3306


In [3]:
def filter_edges_by_bounding_box(edges,max_bb_zone,min_bb_zone):
    #can just use bounding box function to get rid of any inside edges
    filtered_edges = list()

    for i,e in enumerate(edges):
        #print(e)
        if min(e[0][0],e[1][0])>max_bb_zone[0]:
            #print("minx>maxx")
            continue

        if max(e[0][0],e[1][0])<min_bb_zone[0]:
            #print("maxx<minx")
            continue

        if min(e[0][1],e[1][1])>max_bb_zone[1]:
            #print("miny>maxy")
            continue

        if max(e[0][1],e[1][1])<min_bb_zone[1]:
            #print("maxy<miny")
            continue

        if min(e[0][2],e[1][2])>max_bb_zone[2]:
            #print("minz>maxz")
            continue

        if max(e[0][2],e[1][2])<min_bb_zone[2]:
            #print("maxz<minz")
            continue

        filtered_edges.append(e)

    return np.array(filtered_edges)

In [4]:
def sample_edges(filtered_edges,resolution=100):
    
    filtered_edges_distance_lin_alg = [np.linalg.norm(a[0]-a[1]) for a in  filtered_edges]
    
    print("filtered_edges.shape = " + str(np.array(filtered_edges).shape))
    
    midpoints_edges_total = []
    discrete_edges_total = []
    for i,(edg,edg_len) in enumerate(zip(filtered_edges,filtered_edges_distance_lin_alg)):
        #find the number of segments to divide line into
        n_segments = math.ceil(edg_len/resolution)
        discrete_points = (np.linspace(edg[0],edg[1],n_segments+1))
        
        #discrete_edges = np.array((discrete_points[:-1],discrete_points[1:])).T
        discrete_edges = np.vstack((discrete_points[:-1],discrete_points[1:])).reshape(n_segments,2,3)
        discrete_edges_total.append(discrete_edges)
        
        midpoints_edges = [(x[0] + x[1]) / 2 for x in discrete_edges]
        midpoints_edges_total.append(midpoints_edges)
        
    
    #need to reshape
    discrete_edges_total_reshaped = np.vstack(discrete_edges_total)
    midpoints_edges_total_reshaped = np.concatenate(midpoints_edges_total)
    
    return discrete_edges_total_reshaped,midpoints_edges_total_reshaped




In [5]:
#main function that does work
""" pseudocode steps
1) Recieve presyn id and postsyn id
2) Retrieve presyn skeleton
3) Retrieve postsyn soma bounding box and mesh
4) Filter the presyn edges for those that pass through the soma bounding box
5) discretize the edges of presyn even more


"""
def get_somatic_contacts(presyn_id,postsyn_id,threhsold=1500):

    global_start_time = time.time()

    #retrieve presyn skeelton
    presyn_skeleton = (ta3p100.FilteredSkeleton & dict(segmentation=2,segment_id=presyn_id)).fetch1()

    #retrieve postsyn mesh and founding box
    postsyn_soma_data = (ta3p100.FilteredSkeletonMinusSoma & dict(segmentation=2,segment_id=postsyn_id)).fetch1()

    #get the mesh
    soma_vertices = postsyn_soma_data["vertices"]
    bounding_corners = postsyn_soma_data["soma_bounding_corners"]

    start_time = time.time()
    print("edges pulled shape = " + str(presyn_skeleton["edges"].shape))
    #get the filtered edges according to bounding box:
    filtered_edges_presyn = filter_edges_by_bounding_box(presyn_skeleton["edges"],
                                                         bounding_corners[1] + threhsold,
                                                         bounding_corners[0] - threhsold)

    print("extra filter shape = " + str(filtered_edges_presyn.shape))
    print(f"Step 3: filtering edges = {time.time()-start_time}")

    #descretize the edges of the presyn even more
    discrete_edges_total,midpoints_edges_total= sample_edges(filtered_edges_presyn,resolution=50)

    # print(discrete_edges_total.shape) 
    # print(discrete_edges_total)
    # print(midpoints_edges_total.shape)
    # print(midpoints_edges_total)


    # #do KDTree on the midpoints of the new edges to find which ones to keep
    kdtree = KDTree(soma_vertices)

    distances, nearest_nodes = kdtree.query(midpoints_edges_total)

    final_presyn_edges = discrete_edges_total[distances<threhsold]
    print("final_presyn_edges.shape = " + str(final_presyn_edges.shape))

    #Get the final postsyn edges
    """
    1) Map all of the discrete points to the post-synaptic, using query
    2) Construct edge vertices for those
    3) keep all those that are within the threshold

    """


    """ This method is too00000000 slow
    final_postsyn_edges = []
    for ed in final_presyn_edges:
        #print(np.array([ed[0],ed[1]]))
        distances, nearest_nodes = kdtree.query(np.array([ed[0],ed[1]]))
        final_postsyn_edges.append([nearest_nodes[0],nearest_nodes[1]])

    final_postsyn_edges = np.array(final_postsyn_edges)
    """

    distances, nearest_nodes = kdtree.query(final_presyn_edges.reshape(-1,3))
    final_postsyn_edges = soma_vertices[nearest_nodes].reshape(-1,2,3)


    print(f"Total time = {time.time()-global_start_time}")


In [6]:
presyn_id = 648518346341404853 
postsyn_id = 648518346341393609
threhsold = 1500
get_somatic_contacts(presyn_id,postsyn_id,threhsold)

edges pulled shape = (535, 2, 3)
extra filter shape = (121, 2, 3)
Step 3: filtering edges = 0.0054569244384765625
filtered_edges.shape = (121, 2, 3)
final_presyn_edges.shape = (595, 2, 3)
Total time = 0.16702055931091309


In [7]:
# outfile = str(postsyn_id) + "_contacts.npz"
# np.savez(outfile,final_presyn_edges= final_presyn_edges, final_postsyn_edges=final_postsyn_edges,
#          discrete_edges_total=discrete_edges_total,filtered_edges_presyn=filtered_edges_presyn,pre_filter=presyn_skeleton["edges"])

In [8]:
# my_container = np.load("648518346341393609_contacts.npz")
# my_container.files
# my_container["final_presyn_edges"]